<a href="https://colab.research.google.com/github/zhulkhoirr/Steam-Recommendation-System/blob/main/SteamRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistem Rekomendasi Game Pada Platform Steam

## 1. Download Datasets Dari Kaggle

In [ ]:
from google.colab import files
files.upload()

In [1]:
!pip install -q kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d antonkozyriev/game-recommendations-on-steam

Dataset URL: https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam
License(s): CC0-1.0
 98% 647M/660M [00:06<00:00, 127MB/s]
100% 660M/660M [00:06<00:00, 106MB/s]


In [5]:
!unzip -q game-recommendations-on-steam.zip

## 2. Membaca dan Memahami Data

In [6]:
import numpy as np
import pandas as pd

In [7]:
games_df = pd.read_csv('games.csv')

games_df.sample(5)

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
42721,1203042,ONE PIECE: PIRATE WARRIORS 4 Anime Song Pack,2020-03-28,True,False,False,Negative,17,17,16.99,16.99,0.0,True
5251,450820,12 is Better Than 6: The Apostles,2017-12-06,True,True,True,Mixed,48,127,1.24,4.99,75.0,True
31224,1222360,Cybershoes,2020-02-12,True,False,False,Positive,88,17,0.00,0.00,0.0,True
16239,1310460,Half-Life: Alyx Soundtrack,2020-05-05,False,False,False,Very Positive,94,159,9.99,9.99,0.0,True
2241,1336210,The ER: Patient Typhon,2021-04-20,True,False,False,Very Positive,92,50,7.99,7.99,0.0,True


In [8]:
len(games_df)

50872

In [9]:
sort_games_df = games_df.sort_values(by='user_reviews', ascending=False)

sort_games_df.head(10)

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
14398,730,Counter-Strike: Global Offensive,2012-08-21,True,True,True,Very Positive,88,7494460,15.0,0.0,0.0,True
47770,578080,PUBG: BATTLEGROUNDS,2017-12-21,True,False,False,Mixed,57,2217226,0.0,0.0,0.0,True
13176,570,Dota 2,2013-07-09,True,True,True,Very Positive,82,2045628,0.0,0.0,0.0,True
12717,271590,Grand Theft Auto V,2015-04-13,True,False,False,Very Positive,86,1484122,0.0,0.0,0.0,True
14535,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,True,False,False,Very Positive,86,993312,20.0,0.0,0.0,True
47380,440,Team Fortress 2,2007-10-10,True,True,True,Very Positive,93,985819,0.0,0.0,0.0,True
13035,105600,Terraria,2011-05-16,True,True,True,Overwhelmingly Positive,97,943413,10.0,0.0,0.0,True
15363,4000,Garry's Mod,2006-11-29,True,True,True,Overwhelmingly Positive,96,853733,10.0,0.0,0.0,True
13173,252490,Rust,2018-02-08,True,True,False,Very Positive,87,786668,40.0,0.0,0.0,True
14376,1172470,Apex Legends™,2020-11-04,True,False,False,Very Positive,80,713182,0.0,0.0,0.0,True


In [10]:
users_df = pd.read_csv('users.csv')

users_df.sample(5)

,user_id,products,reviews
5633864,12744858,138,1
2494234,1140129,14,1
185749,9927701,24,1
3760410,8986959,82,8
2996068,1585749,21,1


In [11]:
recommendations_df = pd.read_csv('recommendations.csv')

recommendations_df.sample(5)

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
35178128,548570,0,0,2021-04-06,False,29.8,174731,35178128
35886960,1733880,2,0,2022-08-30,False,3.4,1501781,35886960
23872449,287700,0,0,2020-05-10,True,113.8,4934777,23872449
34179154,203160,0,0,2021-06-16,True,7.3,14120137,34179154
7033234,972660,0,0,2022-10-17,True,13.6,11834967,7033234


In [12]:
import json

with open('games_metadata.json', 'r') as file:
    data = file.read()

json_objects = data.split('\n')

data_list = []
for obj in json_objects:
    if obj.strip():
        data_list.append(json.loads(obj))

games_metadata_df = pd.DataFrame(data_list)
games_metadata_df.sample(5)


,app_id,description,tags
41176,909090,A detective must solve increasingly challengin...,"[Movie, Short, Psychological Horror, Puzzle]"
19567,1801640,话题作《精神病大王花》重制作品。 追求精神病少女们的恋爱游戏，你能活到最后吗？,"[Adventure, Horror, Interactive Fiction, Myste..."
10169,1464061,,"[Indie, Casual, Dating Sim, LGBTQ+, Otome, Rom..."
7115,584640,A spy-chicken tries to save the world from doo...,"[Difficult, Arcade, 2D, Action-Adventure, Cute..."
49142,2160330,,"[Action, Gore, Violent]"


## 3. Mempersiapkan Data

In [13]:
recommendations_df = pd.read_csv('recommendations.csv', usecols=['app_id', 'is_recommended', 'user_id'])

recommendations_df.head(5)

,app_id,is_recommended,user_id
0,975370,True,51580
1,304390,False,2586
2,1085660,True,253880
3,703080,True,259432
4,526870,True,23869


In [14]:
n_recommendations = recommendations_df['user_id'].value_counts()
print(n_recommendations)

user_id
11764552    6045
5112758     4152
11656130    3840
5669734     3479
11553593    3392
            ... 
5549796        1
160135         1
10673403       1
4073692        1
2306695        1
Name: count, Length: 13781059, dtype: int64


In [15]:
n_recommendations = recommendations_df['user_id'].value_counts()
recommendations_df = recommendations_df[recommendations_df['user_id'].isin(n_recommendations[n_recommendations >=50].index)].copy()
recommendations_df.head(5)

,app_id,is_recommended,user_id
58,602960,True,737481
138,570940,False,5136277
151,1196590,True,5935019
203,304390,False,10567745
209,244850,True,11143713


In [16]:
print("Jumlah duplikasi:",  recommendations_df.duplicated().sum())

Jumlah duplikasi: 1


In [18]:
recommendations_df.drop_duplicates(inplace=True)
print("Jumlah duplikasi:",  recommendations_df.duplicated().sum())

Jumlah duplikasi: 0


In [17]:
recommendations_df.isnull().sum()

app_id            0
is_recommended    0
user_id           0
dtype: int64

In [19]:
recommendations_df['is_recommended'] = recommendations_df['is_recommended'].astype(int)

## 4. Preprocessing Data

In [20]:
user_ids = recommendations_df['user_id'].unique().tolist()
user2user_encoded = {x:i for i, x in enumerate(user_ids)}
user_encoded2user = {i:x for i, x in enumerate(user_ids)}
recommendations_df['user'] = recommendations_df['user_id'].map(user2user_encoded)
n_users = len(user2user_encoded)

app_ids = recommendations_df['app_id'].unique().tolist()
app2app_encoded = {x:i for i, x in enumerate(app_ids)}
app_encoded2app = {i:x for i, x in enumerate(app_ids)}
recommendations_df['app'] = recommendations_df['app_id'].map(app2app_encoded)
n_app  = len(app2app_encoded)

print("Jumlah user: {}, Jumlah game: {}".format(n_users, n_app))

Jumlah user: 37160, Jumlah game: 36229


In [21]:
recommendations_df

,app_id,is_recommended,user_id,user,app
58,602960,1,737481,0,0
138,570940,0,5136277,1,1
151,1196590,1,5935019,2,2
203,304390,0,10567745,3,3
209,244850,1,11143713,4,4
...,...,...,...,...,...
41154764,1818450,1,13261742,25181,991
41154770,700600,1,6382076,35496,2837
41154771,362960,1,7858713,21724,3024
41154784,633230,0,8107668,18375,2391


In [27]:
recommendations_df = recommendations_df.sample(frac=1, random_state=46)

X = recommendations_df[['user', 'app']].values
y = recommendations_df['is_recommended']

In [28]:
X

array([[30680,  1077],
       [ 7920, 21122],
       [20911,  3007],
       ...,
       [35767, 24180],
       [33440,   449],
       [ 9019,   250]])

In [29]:
y

16910202    1
28772567    1
18831777    1
495301      1
39048057    0
           ..
16389299    1
31836053    1
31918061    1
24754485    1
7731610     1
Name: is_recommended, Length: 3439761, dtype: int64

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

print("Train set is_recommended: {}".format(len(y_train)))
print("Test set is_recommended: {}".format(len(y_test)))

Train set is_recommended: 2751808
Test set is_recommended: 687953


In [33]:
X_train

array([[ 3394,  1120],
       [ 2411,   336],
       [25203, 22946],
       ...,
       [22353,  5088],
       [33071,   702],
       [ 9712,  3579]])

In [34]:
X_train[:, 0]

array([ 3394,  2411, 25203, ..., 22353, 33071,  9712])

In [35]:
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [36]:
X_train_array

[array([ 3394,  2411, 25203, ..., 22353, 33071,  9712]),
 array([ 1120,   336, 22946, ...,  5088,   702,  3579])]

## 5. Membuat Model